PyTorch 1일차 학습

In [2]:
import torch
import pandas as pd
import numpy as np

# --- 1. 가상의 데이터 생성 (csv 파일을 읽는 부분)
# 전세 사기 예측을 위한 간단한 가상 데이터를 만들어봅시다.
# feature: 전세가율(jeonse_ratio), 근저당비율(debt_ratio)
# label: 안전(0), 위험(1)
data = {
    'jeonse_ratio': [0.8, 0.9, 0.7, 0.95, 0.85],
    'debt_ratio': [0.1, 0.3, 0.0, 0.4, 0.2],
    'risk_label': [0, 1, 0, 1, 1]
}

# --- 2. Pandas DataFrame으로 데이터 불러오기
df = pd.DataFrame(data)
print("1. Pandas DataFrame:")
print(df)
print("-"*30)

# --- 3. NumPy 배열로 변환하기
# 모델 학습에는 순수한 숫자 데이터만 필요하므로 .values 속성을 사용
# 예측의 입력이 될 feature와 정답인 label을 분리
features_np = df[['jeonse_ratio', 'debt_ratio']].values
labels_np = df['risk_label'].values

print("2. NumPy Array (Features):")
print(features_np)
print(f"Type: {type(features_np)}")
print("-"*30)

# --- 4. PyTorch 텐서로 변환하기!
# torch.tensor() 함수를 사용하여 NumPy 배열을 PyTorch 텐서로 변환
features_tensor = torch.tensor(features_np, dtype=torch.float32)
labels_tensor = torch.tensor(labels_np, dtype=torch.float32)

print("3. PyTorch Tensor (Features):")
print(features_tensor)
print(f"Type: {type(features_tensor)}")
print(f"Shape: {features_tensor.shape}") # 모양 확인 (5개 데이터, 2개 feature)
print(f"Data Type: {features_tensor.dtype}") # 데이터 타입 확인
print("-" * 30)

# 레이블 텐서도 확인
print("4. PyTorch Tensor (Labels):")
print(labels_tensor)
print(f"Shape: {labels_tensor.shape}") # 모양 확인 (5개 데이터)

# 참고: 레이블 텐서의 차원을 모델 입력에 맞게 [5, 1] 형태로 변경해주면 더 좋습니다.
labels_tensor = labels_tensor.view(-1, 1)
print("\nReshaped Labels Tensor:")
print(labels_tensor)
print(f"Shape: {labels_tensor.shape}") # 모양 확인 ([5, 1])

1. Pandas DataFrame:
   jeonse_ratio  debt_ratio  risk_label
0          0.80         0.1           0
1          0.90         0.3           1
2          0.70         0.0           0
3          0.95         0.4           1
4          0.85         0.2           1
------------------------------
2. NumPy Array (Features):
[[0.8  0.1 ]
 [0.9  0.3 ]
 [0.7  0.  ]
 [0.95 0.4 ]
 [0.85 0.2 ]]
Type: <class 'numpy.ndarray'>
------------------------------
3. PyTorch Tensor (Features):
tensor([[0.8000, 0.1000],
        [0.9000, 0.3000],
        [0.7000, 0.0000],
        [0.9500, 0.4000],
        [0.8500, 0.2000]])
Type: <class 'torch.Tensor'>
Shape: torch.Size([5, 2])
Data Type: torch.float32
------------------------------
4. PyTorch Tensor (Labels):
tensor([0., 1., 0., 1., 1.])
Shape: torch.Size([5])

Reshaped Labels Tensor:
tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.]])
Shape: torch.Size([5, 1])
